In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from numpy.random import randint
from keras.models import Model
from keras.models import Input
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Activation
from keras.layers import LeakyReLU
import tensorflow as tf
import numpy as np
import sys
import matplotlib.pyplot as plt
from numpy import argmax
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
import cv2
from tensorflow.keras.applications import DenseNet121
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras import losses
import os

In [ ]:
path_covid = "/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/covid/"
path_normal = "/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/normal/"

In [ ]:
normal_image = []
covid_image = []
normal_files = os.listdir(path_normal)
covid_files = os.listdir(path_covid)

In [ ]:
print('/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/normal/' + normal_files[0])
print(len(normal_files))
print(len(covid_files))

/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/normal/normal019_90.jpg
3764
1728


In [ ]:
count = 0
for file in normal_files:
  count = count + 1
  if count % 200 == 0:
    print(count)
  temp = path_normal + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  normal_image.append(img)
del normal_files
print(len(normal_image))


count = 0
for file in covid_files:
  count = count + 1
  if count % 200 == 0:
    print(count)
  temp = path_covid + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  covid_image.append(img)
del covid_files
print(len(covid_image))

In [ ]:
ix1 = randint(0, len(normal_image), len(covid_image))
print(ix1.shape)
print(ix1[0])

for i in ix1:
  count = count + 1
  if count % 100 == 0:
    print(count)
  covid_image.append(normal_image[i])
  Y1.append(0)
print(len(covid_image))
del normal_image
del ix1

(1728,)
2575
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
3456


In [ ]:
datagen1 = ImageDataGenerator(featurewise_center=True)
datagen1.fit(covid_image)
mean = datagen1.mean
covid_image = covid_image - mean
print("mean value",mean)
del mean
covid_image = np.reshape(covid_image,(len(covid_image),512,512,3))
print(covid_image.shape)
Y1 = np.array(Y1)

mean value [[[0.01851852 0.01851852 0.01851852]]]
(3456, 512, 512, 3)


In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(covid_image, Y1, test_size=0.2, random_state=42)

In [ ]:
model1 = Sequential([
    DenseNet121(include_top=False, weights='imagenet', input_shape=(512,512,3)),
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(1024,activation='relu'),
    Dense(1, activation='sigmoid')
])
model1.build(input_shape=(512,512,3))
model1.summary()
model1.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.00015, beta_1=0.5), metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 16, 16, 1024)      7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 1024)              1049600   
_________________________________________________________________
module_wrapper_3 (ModuleWrap (None, 1)                 1025      
Total params: 8,088,129
Trainable params: 8,004,481
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
model1.fit(
    X_train1,
    y_train1,
    validation_data=(X_test1,y_test1),
    batch_size=10,
    epochs=6,
    shuffle=True,
)

Epoch 1/6
277/277 [==============================] - 1916s 7s/step - loss: 0.1864 - accuracy: 0.9258 - val_loss: 0.4502 - val_accuracy: 0.8945
Epoch 2/6
277/277 [==============================] - 1844s 7s/step - loss: 0.0809 - accuracy: 0.9707 - val_loss: 0.2025 - val_accuracy: 0.9104
Epoch 3/6
277/277 [==============================] - 1894s 7s/step - loss: 0.0686 - accuracy: 0.9787 - val_loss: 0.0197 - val_accuracy: 0.9928
Epoch 4/6
277/277 [==============================] - 1855s 7s/step - loss: 0.0156 - accuracy: 0.9946 - val_loss: 6.8231e-04 - val_accuracy: 1.0000
Epoch 5/6
277/277 [==============================] - 1855s 7s/step - loss: 0.0500 - accuracy: 0.9870 - val_loss: 0.0249 - val_accuracy: 0.9928
Epoch 6/6
277/277 [==============================] - 1844s 7s/step - loss: 0.0383 - accuracy: 0.9863 - val_loss: 0.0049 - val_accuracy: 1.0000


In [ ]:
model1.save_weights("/content/drive/MyDrive/IndividualProject/SPGC-COVID/DesNet-covid-normal.ckpt")
del covid_image

In [ ]:
model2 = Sequential([
    DenseNet121(include_top=False, weights='imagenet', input_shape=(512,512,3)),
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(1024,activation='relu'),
    Dense(1, activation='sigmoid')
])

29089792/29084464 [==============================] - 1s 0us/step


In [ ]:
model2.load_weights("/content/drive/MyDrive/IndividualProject/SPGC-COVID/COVID_DenseNet/DesNet-covid-normal.ckpt")

In [ ]:
img = np.array(X_train1[0])
img = np.reshape(img,(1,512,512,3))

In [ ]:
print(model1(img))

tf.Tensor([[0.00039983]], shape=(1, 1), dtype=float32)


In [ ]:
print(model2(img))

tf.Tensor([[0.00039983]], shape=(1, 1), dtype=float32)
